In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('processed_data_all.csv')
data.head(10)

,content
0,Kumaha barudak? Well!Yuk ramaikan Pesta Anak N...
1,Kustini Sri Purnomo memiliki strategi jitu unt...
2,"Peraih Emas Olimpiade Tokyo, Greysia Polii bic..."
3,Kapolri Jenderal Lisyto Sigit Prabowo bicara a...
4,"Menteri BUMN, Erick Thohir bicara bangsa Indon..."
5,Iring-iringan mobil SUV pasukan pengawal Presi...
6,Seorang remaja Palestina umur 14 tahun ditemba...
7,"Ketua Umum PDIP, Megawati Soekarnoputri mengin..."
8,Sekjen PDIP Hasto Kristiyanto berbicara terkai...
9,Presiden Joko Widodo memimpin upacara peringat...


In [3]:
len(data)

27390

In [4]:
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline

# Load model analisis sentimen bahasa Indonesia dengan tiga kelas sentimen
# sentiment_analysis = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")


2024-04-19 17:12:13.800777: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-19 17:12:15.341907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# summarization_pipe = pipeline("summarization", model="cahya/bert2gpt-indonesian-summarization")

In [6]:
# summarization_pipe = pipeline("text-classification", model="taufiqdp/indonesian-sentiment")

In [7]:
# summarization_pipe = pipeline("text-classification", model="crypter70/IndoBERT-Sentiment-Analysis")

In [9]:
sentimen_pipe = pipeline("sentiment-analysis", model="indobenchmark/indobert-large-p2")

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/indobenchmark/indobert-large-p2/adb98dec62ea217e729579642017fde2aa41c87a19adfd0da4049de676190533?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1713780796&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzc4MDc5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9pbmRvYmVuY2htYXJrL2luZG9iZXJ0LWxhcmdlLXAyL2FkYjk4ZGVjNjJlYTIxN2U3Mjk1Nzk2NDIwMTdmZGUyYWE0MWM4N2ExOWFkZmQwZGE0MDQ5ZGU2NzYxOTA1MzM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=hhHvc2l25dRTrQHsL1BcaiZZ75J1ygPMBaOEevsLSO6SCmAQKhiVwqM5gtyQeFYMUEbozpCCey9m9Vo2-zx1G%7EVvR-guxqz7POcXKmA926f4Ws4uPktqpRUTU6m9tDqzhhqCGyacWgen7Cd3Zvphtlm5TyhZptWqsilc2k6I0IiCIbwVXO1oSn7vvADg9DHiCtJDdgfcQRWmZ8BAr2ZVb87VqO0Vm7Pgjd4AEeIBufyC5ka5iRv1Pl6FCQa6gfD99q4FJjxw

pytorch_model.bin:   8%|7         | 105M/1.34G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/indobenchmark/indobert-large-p2/adb98dec62ea217e729579642017fde2aa41c87a19adfd0da4049de676190533?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1713780796&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzc4MDc5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9pbmRvYmVuY2htYXJrL2luZG9iZXJ0LWxhcmdlLXAyL2FkYjk4ZGVjNjJlYTIxN2U3Mjk1Nzk2NDIwMTdmZGUyYWE0MWM4N2ExOWFkZmQwZGE0MDQ5ZGU2NzYxOTA1MzM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=hhHvc2l25dRTrQHsL1BcaiZZ75J1ygPMBaOEevsLSO6SCmAQKhiVwqM5gtyQeFYMUEbozpCCey9m9Vo2-zx1G%7EVvR-guxqz7POcXKmA926f4Ws4uPktqpRUTU6m9tDqzhhqCGyacWgen7Cd3Zvphtlm5TyhZptWqsilc2k6I0IiCIbwVXO1oSn7vvADg9DHiCtJDdgfcQRWmZ8BAr2ZVb87VqO0Vm7Pgjd4AEeIBufyC5ka5iRv1Pl6FCQa6gfD99q4FJjxw

pytorch_model.bin:  94%|#########3| 1.26G/1.34G [00:00<?, ?B/s]

In [ ]:
# # Fungsi untuk melabeli sentimen pada setiap baris data
# def label_sentiment(text):
#     # Analisis sentimen menggunakan model Transformer
#     result = sentiment_analysis(text)
#     # Ambil label sentimen teratas
#     top_sentiment = result[0]
#     label = top_sentiment['label']
#     score = top_sentiment['score']
#     print(label)
#     return label, score

# # Fungsi untuk melabeli sentimen menggunakan multi-threading
# def label_sentiment_multithread(text_list):
#     with ThreadPoolExecutor() as executor:
#         results = executor.map(label_sentiment, text_list)
#     return list(results)

# # Bagi data menjadi batch agar dapat diproses menggunakan multi-threading
# batch_size = 1000
# num_batches = len(data) // batch_size + 1
# batches = [data[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# # Melabeli sentimen untuk setiap batch menggunakan multi-threading
# labeled_data = []
# for batch in batches:
#     texts = batch['content'].tolist()
#     labels = label_sentiment_multithread(texts)
#     labeled_data.extend(labels)

# # Memasukkan hasil label sentimen ke dalam DataFrame
# data['sentiment_label'], data['sentiment_score'] = zip(*labeled_data)

In [ ]:
# Simpan data yang telah dilabeli ke dalam file CSV
data.to_csv('label_all.csv', index=False)